# fMRI - individual resting state preprocessing (AROMA)
### Adapted from Emma Mallas PTA fMRI notebook
E. Mallas Jan 2018 (e.mallas@imperial.ac.uk)  
N. Bourke Nov 2019 (n.bourke@imperial.ac.uk)  

Last edit ~ 01/05/2020  
for paeds project

#### ~ Adaptations:  
* Source BIDS format input
* Change to python kernal accounting for change in Jupyter
* (Could include FreeSurfer instead of BET)

#### ~ Questions:
* Matlab motion parameter script


# Dependency: Pre run BIDS script 
* Download, convert and organise structure of raw data

# Preprocessing Pipeline

1. Setup dependencies & project directories
2. Skull stripping using BET in FSL
3. Lower level FEAT (without high-pass filtering because of ICA AROMA) - the preprocessing tabs
        - motion correction
        - slice timing correction
        - spatial smoothing
        - registration
4. Create functional masks
5. Run matlab script to calculate 24 motion parameters = .txt file to include in stage 8 (ORDER OF THIS??)
6. Identifying motion, artifacts, noise components 
        5a. ICA AROMA
        5b. Motion outliers   
7. Denoising using components
8. Regress out white matter, CSF and motion paramters (6 that came from MCFLIRT +18)
9. Lower level FEAT of cleaned data (with high-pass filtering) - the preprocessing tabs

=========================================================================


## 1) Initial setup: Define data directories

In [1]:
import os

project = "PTBI" 

directory = ("/rds/general/project/c3nl_djs_imaging_data/ephemeral/" + project + "/data/")
if not os.path.exists(directory):
    os.makedirs(directory)
    
raw = ("/rds/general/project/c3nl_djs_imaging_data/live/data/raw/" + project + "/")
source = ("/rds/general/project/c3nl_djs_imaging_data/live/data/sourcedata/")
workingDir = ("/rds/general/project/c3nl_djs_imaging_data/ephemeral/" + project + "/")
setup = (workingDir + "/rest_setup.sh" )


In [2]:
%%writefile $setup

$(: Project label) 
project="PTBI"

$(:  Where is the RAW directory?)
export raw=/rds/general/project/c3nl_djs_imaging_data/live/data/raw/${project};
export source=/rds/general/project/c3nl_djs_imaging_data/live/data/sourcedata/

$(: dependencies)
export dep=/rds/general/project/c3nl_shared/live/dependencies/
export templates=/rds/general/project/c3nl_djs_imaging_data/live/analysis/paeds/templates
export workingDir=/rds/general/project/c3nl_djs_imaging_data/ephemeral/${project}

# Define modules    
fsl="module load fsl/6.0.1"; # 5.0.10 not running feats

Overwriting /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI//rest_setup.sh


## 2) Skull strip T1s

#####  Brain extraction tool (BET) in FSL 
 - bet [input] [output] [options]

> -m generate binary brain mask

> -f fractional intensity threshold (0->1); default=0.5; smaller values give larger brain outline estimates

 - Rename the skull stripped T1 as t1_brain.nii.gz - this will be required for future steps. 


In [13]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $project

# Set empty command File 
echo -n "" > ${workingDir}/commandLogs/${project}_bet_script.txt
job=${workingDir}/commandLogs/${project}_bet_script.txt

#~~~~~~~~~~~~~~~~~~~~~#
# Setup job - loop over subjects & visits
#~~~~~~~~~~~~~~~~~~~~~#

for sub in `ls ${raw}`
do
    for ses in `ls ${source}/sub-${sub}`
    do
        # Set job variables
        input=${source}/sub-${sub}/${ses}/anat/T1w/sub-${sub}_${ses}_T1w.nii
        mkdir -p ${workingDir}/data/sub-${sub}/${ses}/anat/T1w/
        output=${workingDir}/data/sub-${sub}/${ses}/anat/T1w/sub-${sub}_${ses}_T1w_brain.nii
        thr=0.2
        
        # Add jobs to submission textfile
        echo "${fsl}; bet ${input} ${output} -R -Z -m -B -f ${thr}" >> ${job}
        
    done
done;

#~~~~~~~~~~~~~~~~~~~~~#
# Run job
#~~~~~~~~~~~~~~~~~~~~~#

    ${dep}/hpcSubmit ${job} 02:00:00 1 8Gb
    echo ""; echo "***"; echo ""; echo "Head submitted commands:"
    head ${job}


PTBI
input is = /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/commandLogs/PTBI_bet_script.txt
Walltime = 02:00:00
Number of CPUs = 1
Memory = 8Gb
Array jobs submitted: 60
Job submitted: Fri  1 May 06:12:31 BST 2020
1419439[].pbs

***

Head submitted commands:
module load fsl; bet /rds/general/project/c3nl_djs_imaging_data/live/data/sourcedata//sub-CIF0537/ses-2017-03-27-PILOT_AMR/anat/T1w/sub-CIF0537_ses-2017-03-27-PILOT_AMR_T1w.nii /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/source/sub-CIF0537/ses-2017-03-27-PILOT_AMR/anat/T1w/sub-CIF0537_ses-2017-03-27-PILOT_AMR_T1w_brain.nii -R -Z -m -B -f 0.2
module load fsl; bet /rds/general/project/c3nl_djs_imaging_data/live/data/sourcedata//sub-CIF1638/ses-2017-03-30-PTBI001/anat/T1w/sub-CIF1638_ses-2017-03-30-PTBI001_T1w.nii /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/source/sub-CIF1638/ses-2017-03-30-PTBI001/anat/T1w/sub-CIF1638_ses-2017-03-30-PTBI001_T1w_brain.nii -R -Z -m -B -f 0.2
module load fsl; bet

##### ! QC STEP REQUIRED:
    - Check each subject to ensure no brain tissue has been eroded and skull is fully stripped. 
    - If these problems occur then try again with a different -f BET parameter. 
    - BET parameters can be changed on a subject by subject basis to ensure appropriate skull stripping for each subject. 

Concatenate all of the t1_brain images into one 4d file to scroll through and check BET output (where there is one volume per subject)

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $project
${fsl}
#~~~~~~~~~~~~~~~~~~~~~#
# Job
#~~~~~~~~~~~~~~~~~~~~~#
echo -n "" > ${workingDir}/betsubjects.txt
for sub in `ls ${workingDir}/data/`
do 
    for ses in `ls ${workingDir}/data/${sub}`
    do
        echo "${workingDir}/data/${sub}/${ses}/anat/T1w/${sub}_${ses}_T1w_brain.nii.gz" >> ${workingDir}/commandLogs/betsubjects.txt
    done;
done

mkdir -p ${workingDir}/QC
fslmerge -t ${workingDir}/QC/allsubjects_t1_brain.nii.gz $(cat ${workingDir}/commandLogs/betsubjects.txt)


#### If a freeSurfer segmentation exists (end of volume pipeline), convert the brain extraction to .nii

In [ ]:
mri_convert [options] <in volume> <out volume>

ephemeral/PTBI/data/sub-xx/ses-xx/anat/T1w/fs/mri/brain.mgz & brainmask.mgz

## 3) Lower Level FEAT

A lower-level FEAT is first-level analysis for analysing each session's data - i.e. the time-series analysis of the raw 4D FMRI data. This is set up for one subject using the FSL GUI, saved as a fsf template and then applied to all subjects. 

In the PTA analysis, as ICA-AROMA will be used the first lower level FEAT will not highpass filter the data - this will be done after noise cleaning nor run a melodic.

### Create template FSF file

Create a template of the .fsf file that will be used as in input for the FEATs. This is where all the preprocessing parameters are set. Change as required depending on individual requirements (see note above re: FIX / ICA AROMA)

You have two options for creating a template.

1. Use the GUI and make your own settings but include these strings in these places (you do this for one subject then you will clone it in the next step). Ensure you keep the following strings in the same places as in the example below. (e.g. 

2. Use the cell below to add desired options - this achieves the same aim as going through the GUI. As before it will then be cloned for each subject in the next step. 

In each case you will need to include the following strings (they will be defined in the next step):


| Key            | Value
|:---------------|:--------------|
|``__MYFMRI__``  | Functional image |
|``__MYHIGHRES__``  | Skull-stripped T1 image |
|``__MYOUTPUTDIR__``  | Path to directory you want the resulting file to be output to |
|``__MYTR__``  | The repetition time (get this from scan header info) |
|``__MYNPTS__``  | Number of volumes (get this from scan header info) |

You need to make a note of where you save this file - I have a deidcated directory in /analysis/templates for various different FEAT templates (i.e. the pre and post ICA AROMA feats)
        


In [21]:
%%file /rds/general/project/c3nl_djs_imaging_data/live/analysis/paeds/templates/preICA_feat.fsf

# FEAT version number
set fmri(version) 6.00

# Are we in MELODIC?
set fmri(inmelodic) 0

# Analysis level
# 1 : First-level analysis
# 2 : Higher-level analysis
set fmri(level) 1

# Which stages to run
# 0 : No first-level analysis (registration and/or group stats only)
# 7 : Full first-level analysis
# 1 : Pre-processing
# 2 : Statistics
set fmri(analysis) 1

# Use relative filenames
set fmri(relative_yn) 0

# Balloon help
set fmri(help_yn) 1

# Run Featwatcher
set fmri(featwatcher_yn) 1

# Cleanup first-level standard-space images
set fmri(sscleanup_yn) 0

# Output directory
set fmri(outputdir) "__MYOUTPUTDIR__"

# TR(s)
set fmri(tr) "__MYTR__"

# Total volumes
set fmri(npts) "__MYNPTS__"

# Delete volumes
set fmri(ndelete) 0

# Perfusion tag/control order
set fmri(tagfirst) 1

# Number of first-level analyses
set fmri(multiple) 1

# Higher-level input type
# 1 : Inputs are lower-level FEAT directories
# 2 : Inputs are cope images from FEAT directories
set fmri(inputtype) 2

# Carry out pre-stats processing?
set fmri(filtering_yn) 1

# Brain/background threshold, %
set fmri(brain_thresh) 10

# Critical z for design efficiency calculation
set fmri(critical_z) 5.3

# Noise level
set fmri(noise) 0.66

# Noise AR(1)
set fmri(noisear) 0.34

# Motion correction
# 0 : None
# 1 : MCFLIRT
set fmri(mc) 1

# Spin-history (currently obsolete)
set fmri(sh_yn) 0

# B0 fieldmap unwarping?
set fmri(regunwarp_yn) 0

# GDC Test
set fmri(gdc) ""

# EPI dwell time (ms)
set fmri(dwell) 0.7

# EPI TE (ms)
set fmri(te) 35

# % Signal loss threshold
set fmri(signallossthresh) 10

# Unwarp direction
set fmri(unwarp_dir) y-

# Slice timing correction
# 0 : None
# 1 : Regular up (0, 1, 2, 3, ...)
# 2 : Regular down
# 3 : Use slice order file
# 4 : Use slice timings file
# 5 : Interleaved (0, 2, 4 ... 1, 3, 5 ... )
set fmri(st) 5

# Slice timings file
set fmri(st_file) ""

# BET brain extraction
set fmri(bet_yn) 1

# Spatial smoothing FWHM (mm)
set fmri(smooth) 6.0

# Intensity normalization
set fmri(norm_yn) 0

# Perfusion subtraction
set fmri(perfsub_yn) 0

# Highpass temporal filtering
set fmri(temphp_yn) 0

# Lowpass temporal filtering
set fmri(templp_yn) 0

# MELODIC ICA data exploration
set fmri(melodic_yn) 0

# Carry out main stats?
set fmri(stats_yn) 0

# Carry out prewhitening?
set fmri(prewhiten_yn) 1

# Add motion parameters to model
# 0 : No
# 1 : Yes
set fmri(motionevs) 0
set fmri(motionevsbeta) ""
set fmri(scriptevsbeta) ""

# Robust outlier detection in FLAME?
set fmri(robust_yn) 0

# Higher-level modelling
# 3 : Fixed effects
# 0 : Mixed Effects: Simple OLS
# 2 : Mixed Effects: FLAME 1
# 1 : Mixed Effects: FLAME 1+2
set fmri(mixed_yn) 2

# Higher-level permutations
set fmri(randomisePermutations) 5000

# Number of EVs
set fmri(evs_orig) 1
set fmri(evs_real) 2
set fmri(evs_vox) 0

# Number of contrasts
set fmri(ncon_orig) 1
set fmri(ncon_real) 1

# Number of F-tests
set fmri(nftests_orig) 0
set fmri(nftests_real) 0

# Add constant column to design matrix? (obsolete)
set fmri(constcol) 0

# Carry out post-stats steps?
set fmri(poststats_yn) 0

# Pre-threshold masking?
set fmri(threshmask) ""

# Thresholding
# 0 : None
# 1 : Uncorrected
# 2 : Voxel
# 3 : Cluster
set fmri(thresh) 3

# P threshold
set fmri(prob_thresh) 0.05

# Z threshold
set fmri(z_thresh) 3.1

# Z min/max for colour rendering
# 0 : Use actual Z min/max
# 1 : Use preset Z min/max
set fmri(zdisplay) 0

# Z min in colour rendering
set fmri(zmin) 2

# Z max in colour rendering
set fmri(zmax) 8

# Colour rendering type
# 0 : Solid blobs
# 1 : Transparent blobs
set fmri(rendertype) 1

# Background image for higher-level stats overlays
# 1 : Mean highres
# 2 : First highres
# 3 : Mean functional
# 4 : First functional
# 5 : Standard space template
set fmri(bgimage) 1

# Create time series plots
set fmri(tsplot_yn) 1

# Registration to initial structural
set fmri(reginitial_highres_yn) 0

# Search space for registration to initial structural
# 0   : No search
# 90  : Normal search
# 180 : Full search
set fmri(reginitial_highres_search) 90

# Degrees of Freedom for registration to initial structural
set fmri(reginitial_highres_dof) 3

# Registration to main structural
set fmri(reghighres_yn) 1

# Search space for registration to main structural
# 0   : No search
# 90  : Normal search
# 180 : Full search
set fmri(reghighres_search) 90

# Degrees of Freedom for registration to main structural
set fmri(reghighres_dof) 7

# Registration to standard image?
set fmri(regstandard_yn) 1

# Use alternate reference images?
set fmri(alternateReference_yn) 0

# Standard image
set fmri(regstandard) "/apps/fsl/5.0.10/fsl/data/standard/MNI152_T1_2mm_brain"

# Search space for registration to standard space
# 0   : No search
# 90  : Normal search
# 180 : Full search
set fmri(regstandard_search) 90

# Degrees of Freedom for registration to standard space
set fmri(regstandard_dof) 12

# Do nonlinear registration from structural to standard space?
set fmri(regstandard_nonlinear_yn) 0

# Control nonlinear warp field resolution
set fmri(regstandard_nonlinear_warpres) 10 

# High pass filter cutoff
set fmri(paradigm_hp) 100

# Total voxels
set fmri(totalVoxels) 30965760


# Number of lower-level copes feeding into higher-level analysis
set fmri(ncopeinputs) 0

# 4D AVW data or FEAT directory (1)
set feat_files(1) "__MYFMRI__"

# Add confound EVs text file
set fmri(confoundevs) 0

# Subject's structural image for analysis 1
set highres_files(1) "__MYHIGHRES__"

# EV 1 title
set fmri(evtitle1) ""

# Basic waveform shape (EV 1)
# 0 : Square
# 1 : Sinusoid
# 2 : Custom (1 entry per volume)
# 3 : Custom (3 column format)
# 4 : Interaction
# 10 : Empty (all zeros)
set fmri(shape1) 0

# Convolution (EV 1)
# 0 : None
# 1 : Gaussian
# 2 : Gamma
# 3 : Double-Gamma HRF
# 4 : Gamma basis functions
# 5 : Sine basis functions
# 6 : FIR basis functions
set fmri(convolve1) 2

# Convolve phase (EV 1)
set fmri(convolve_phase1) 0

# Apply temporal filtering (EV 1)
set fmri(tempfilt_yn1) 1

# Add temporal derivative (EV 1)
set fmri(deriv_yn1) 1

# Skip (EV 1)
set fmri(skip1) 0

# Off (EV 1)
set fmri(off1) 30

# On (EV 1)
set fmri(on1) 30

# Phase (EV 1)
set fmri(phase1) 0

# Stop (EV 1)
set fmri(stop1) -1

# Gamma sigma (EV 1)
set fmri(gammasigma1) 3

# Gamma delay (EV 1)
set fmri(gammadelay1) 6

# Orthogonalise EV 1 wrt EV 0
set fmri(ortho1.0) 0

# Orthogonalise EV 1 wrt EV 1
set fmri(ortho1.1) 0

# Contrast & F-tests mode
# real : control real EVs
# orig : control original EVs
set fmri(con_mode_old) orig
set fmri(con_mode) orig

# Display images for contrast_real 1
set fmri(conpic_real.1) 1

# Title for contrast_real 1
set fmri(conname_real.1) ""

# Real contrast_real vector 1 element 1
set fmri(con_real1.1) 1

# Real contrast_real vector 1 element 2
set fmri(con_real1.2) 0

# Display images for contrast_orig 1
set fmri(conpic_orig.1) 1

# Title for contrast_orig 1
set fmri(conname_orig.1) ""

# Real contrast_orig vector 1 element 1
set fmri(con_orig1.1) 1

# Contrast masking - use >0 instead of thresholding?
set fmri(conmask_zerothresh_yn) 0

# Do contrast masking at all?
set fmri(conmask1_1) 0

##########################################################
# Now options that don't appear in the GUI

# Alternative (to BETting) mask image
set fmri(alternative_mask) ""

# Initial structural space registration initialisation transform
set fmri(init_initial_highres) ""

# Structural space registration initialisation transform
set fmri(init_highres) ""

# Standard space registration initialisation transform
set fmri(init_standard) ""

# For full FEAT analysis: overwrite existing .feat output dir?
set fmri(overwrite_yn) 0


Overwriting /rds/general/project/c3nl_djs_imaging_data/live/analysis/paeds/templates/preICA_feat.fsf


##### Individualise copies of the template .fsf

In [3]:
!module load fix_unwritable_tmp # prob not required anymore, was a patch for feat when couldnt write to tmp files

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo ${workingDir}

#~~~~~~~~~~~~~~~~~~~~~#
# Job
#~~~~~~~~~~~~~~~~~~~~~#

# Path to the template .fsf file (defined above)
template_fsf=${templates}/preICA_feat.fsf;

# Blank commands file for parallelise
echo -n "" > ${workingDir}/commandLogs/clone_feat_template.txt
job=${workingDir}/commandLogs/clone_feat_template.txt 

for sub in `ls ${raw}`
do
    for ses in `ls ${source}/sub-${sub}`
    do
        
        # Make a .fsf file which will be individualised to this subject/visit
        mkdir -p ${workingDir}/data/sub-${sub}/${ses}/func/rest
        fsffile="${workingDir}/data/sub-${sub}/${ses}/func/rest/sub-${sub}_${ses}_preICA_feat.fsf"
        
        # Copy the template onto this temporary file
        cp ${template_fsf} ${fsffile} 
    
        fmriInput=${source}/sub-${sub}/${ses}/func/rest/sub-${sub}_${ses}_rest.nii
        t1Input=${workingDir}/data/sub-${sub}/${ses}/anat/T1w/sub-${sub}_${ses}_T1w_brain.nii.gz
        #echo $fmriInput
        #echo $t1Input
        
        # This block will replace all the keys in map[key]=value that occur in the .fsf file
        # with their corresponding values 
        declare -A map=()
        map[__MYFMRI__]="${fmriInput}"
        map[__MYHIGHRES__]="${t1Input}"
        map[__MYOUTPUTDIR__]="${workingDir}/data/sub-${sub}/${ses}/func/rest/preICA"
        map[__MYTR__]=`fslinfo ${map[__MYFMRI__]} | grep ^pixdim4 | awk '{print $2}'`
        map[__MYNPTS__]=`fslnvols ${map[__MYFMRI__]}`
       
        for j in "${!map[@]}"
        do
            from=$j; to=${map[$j]};
            sed -i "s|$from|$to|g" ${fsffile}
        done
        
        echo "${fsl}; feat ${fsffile}" >> ${job} #${fsl} 
    done
done;

# pause, check feat files

#~~~~~~~~~~~~~~~~~~~~~#
# Run job
#~~~~~~~~~~~~~~~~~~~~~#

    ${dep}/hpcSubmit ${job} 02:00:00 1 8Gb
    echo ""; echo "***"; echo ""; echo "Head submitted commands:"
    head ${job}


You can view the progress of the FEATS by navigating to the FEAT directory in a terminal and viewing the report.html log in firefox
* Check Emmas notebook/Emma if there are difficulties with registration (DOF/BBR)

## 4) Create functional masks

Once the FEATS have finished running and you are happy with the registration you can move on to create the functional masks. 

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $project

#~~~~~~~~~~~~~~~~~~~~~#
# Job
#~~~~~~~~~~~~~~~~~~~~~#

echo -n "" > ${workingDir}/func_bet.txt

for sub in `ls ${workingDir}/source/`
do
    for ses in `ls ${workingDir}/source/${sub}`
    do  
        echo "${fsl}; bet ${workingDir}/source/${sub}/${ses}/preICA.feat/example_func.nii.gz ${workingDir}/source/${sub}/${ses}/preICA.feat/example_func_brain.nii.gz -f 0.3 -g 0 -R -m " >> ${workingDir}/func_bet.txt 
    done
done;

job=${workingDir}/func_bet.txt

#~~~~~~~~~~~~~~~~~~~~~#
# Run job
#~~~~~~~~~~~~~~~~~~~~~#

    ${dep}/hpcSubmit ${job} 02:00:00 1 8Gb
    echo ""; echo "***"; echo ""; echo "Head submitted commands:"
    head ${job}


## 5)MOTION - MATLAB

Head movement in the scanner and the associated motion-induced changes in the images are a big problem in fMRI. Why?
Changes in signal due to the BOLD effect are quite weak (around 1%), but changes in intensity induced by motion are much larger, even for very small movements. We could therefore be seeing false activation (attributed to motion).

Head motion affects the data in many different ways which can be largely separated into first and higher order motion effects:
<br>
- First-order effect of motion is simply seen as the spatial misalignment from one volume to the next. First-order effects of motion are corrected using motion correction.
<br>
- Higher-order motion effects are less intuitive to understand and harder to remove from the data. They are effects of head motion that impact subsequent analyes even after the displacemnets of the individual volumes have been corrected for.  These relate to *partial volume effects, spin history and varying field inhomogeneities.*
    - ***Partial volume effects:*** result from the fact that voxels at boundaries between tissue types (e.g. GM, WM, CSF) sampled during fMRI are often made up of more than one type of tissue (i.e. there is both GM and WM in one voxel, the voxel crosses the boundary). When head motion occurs the relative contribution of these tissue types may change because the same voxel is now in a different position in the brain. These partial volume changes can casue secondary changes i the local BOLD signal which are only approximately corrected by motion correction. 
    - *** Spin history effects: *** occur that relate to the fact that the TR in fMRI is too short to allow the magnetic state of the nuclei to completley "forget" about the previous excitation. As long as a region is excited at regular intervals this isn't a problem, but when a region move from one slice to the next due to the head motion, the nuclei in that region will be excited slightly earlier or later than they should be (i.e their next 'due' excitation). Therefore the "history" of the spin excitation differs between nuclei that have not moved slices and those that have. 
    - *** Field inhomogeneities: ***in the static magnetic field occur close to pockets of air (e.g. around the nasal sinuses) and are corrected for with shimming and fieldmaps. When head motion occurs the field inhomogeneitis change, adn therefore the shimming and fieldmaps become suboptimal. As such, complex interactions between motion and dropout can occur, adding localised signal fluctuations. 
<br>

Because motion is such a large contributor to structured noise it's important to clean the data thoroughly for motion artefact. Motion correction (e.g. MCFLIRT) works by spatially registering each volume separately to a chosen reference volume (i.e. making sure they overlap correctly. Motion correction is one of the first preprocessing steps done - in this pipeline is incoproated during the first lower level FEAT. Motion correction applies transformations to each volume such that all resulting volumes are mathced up spatially. In addition to aligning the volumes, motion correction also procides outputs of head motion estimates throughout the scan known as **motion parameters**. There are 6 motion parameters; three describe translation in three directions (left-right, up-down, back-front), three describe rotational movements (pitch, yaw, roll). These motion parameters can be used later in preprocessing. 
    
#### There are 4 ways to deal with motion:

1. MCFLIRT (done during the intiial preprocessing)
2. FD AND DVARS (removal of subjects based on (arbitrarily define) thresholds)
3. ICA DENOISING (removal of 'noise' components)
4. REGRESS OUT 24 MOTION PARAMTERS (using fsl_regfilt - can be in same script as regressing out WM and CSF) 


  
* > Need to report motion between groups, this can be done from the output of FD
* > Record how many components you remove for subjects (+avg) & On average how many components are removed and if this differes between groups. 
* > 3mm per volume, or >15% of vol excedded 0.5mm



TO DO: CHECK ON FSL MAILING LIST IF REGRESSING OUT MOTION PARAMETERS AFTER ICA AROMA IS RECOMMENDED / OK / ADVISABLE ETC. (SARA THINKS PROBABLY DO DO THIS)

       [POST-HOC]
        can look at mean FD values per subject and how this correlates with mean whole brain functional connectivity (or hopefully does not correlate) - can look at this across patient groups

### Framewise displacement FD

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $project

#~~~~~~~~~~~~~~~~~~~~~#
# Job
#~~~~~~~~~~~~~~~~~~~~~#

#The tool fsl_motion_outliers is designed to detect timepoints in an fMRI dataset that have been corrupted by large motion
# Usage: fsl_motion_outliers -i <input 4D image> -o <output confound file> [options]

echo -n "" > ${workingDir}/motion_outliers.txt
for sub in `ls ${rawDir}`
do
    for ses in `ls ${rawDir}/${sub}`
    do
    
    fmriInput=${rawDir}/${sub}/${ses}/func/rest/${sub}_${ses}_rest.nii
    outDir=${workingDir}/source/${sub}/${ses}/func/rest/
    
    # fsl_motion_outliers -i ${fmriInput} -o ${outDir}/QC/${sub}_${ses}_DVARS --dvars --thresh=50 -s ${outDir}/QC/${sub}_${ses}_DVARS.txt -p ${outDir}/QC/${sub}_${ses}_DVARS -v ${outDir}/QC/${sub}_${ses}_DVARS_output.txt #>> ${workingDir}/motion_outliers.txt   
    echo "${fsl}; fsl_motion_outliers  -i ${fmriInput} -o ${outDir}/QC/${sub}_${ses}_FD --fd --thresh=0.5 -s ${outDir}/QC/${sub}_${ses}_FD.txt -p ${outDir}/QC/${sub}_${ses}_FD -v ${outDir}/QC/${sub}_${ses}_FD_output_thresh05.txt" >> ${workingDir}/motion_outliers.txt   
    done
done
    
    job=${workingDir}/motion_outliers.txt
    
#~~~~~~~~~~~~~~~~~~~~~#
# Run job
#~~~~~~~~~~~~~~~~~~~~~#

    ${dep}/hpcSubmit ${job} 01:00:00 1 4Gb
    echo ""; echo "***"; echo ""; echo "Head submitted commands:"
    head ${job}


## 6) ICA-AROMA

ICA-AROMA is a data-driven method to identify and remove motion-related ICA components from FMRI data. It exploits a small, but robust set of theoretically motivated features specifically aimed at selecting motion-related components from FSL-MELODIC output. Unlike FSL-FIX it is not aimed at more generic ICA_based denoising but due to the robustness of the feature set does not require classifier re-training. This beta-version package requires installation of Python2.7 and FSL. Read the provided 'Manual.pdf' on the FSL page for more details

ICA-AROMA assumes that you ran non-linear registration when using the -feat option. I have used linear registration and therefore do not have the highres2standard_warp.nii.gz file that ICA-AROMA wants when using the -feat argument input. In order to overcome this I need to run ICA-AROMA with the affine registration file, specifying the -affmat argument instead of the -warp argument. A mask of the functional data is needed to do this (make sure you have created the functional masks in the previous step).

1. Strip functional data and create a mask.
2. Run ICA-AROMA with the alternative arguments. 

### Create a python env with the dependencies to run ICA-AROMA
#### This can then be sourced when running the job
##### When created don't run this cell again

In [ ]:
# %%script bash

# source activate aroma
# conda create -n aroma python=2.7 numpy pandas matplotlib seaborn scipy future

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $project

ICAAROMA=~/c3nl_tools/ICA-AROMA-master/ICA_AROMA.py
aroma="source activate aroma"

#~~~~~~~~~~~~~~~~~~~~~#
# Job
#~~~~~~~~~~~~~~~~~~~~~#

echo -n "" > ${workingDir}/ICA_AROMA.txt
    
for sub in `ls ${workingDir}/source/`
do
    for ses in `ls ${workingDir}/source/${sub}`
    do 
        echo "${fsl}; ${aroma}; python2.7 ${ICAAROMA} -i ${workingDir}/source/${sub}/${ses}/preICA.feat/filtered_func_data.nii.gz -o ${workingDir}/source/${sub}/${ses}/preICA.feat/ICA_AROMA -mc ${workingDir}/source/${sub}/${ses}/preICA.feat/mc/prefiltered_func_data_mcf.par -a ${workingDir}/source/${sub}/${ses}/preICA.feat/reg/example_func2standard.mat -m ${workingDir}/source/${sub}/${ses}/preICA.feat/example_func_brain_mask.nii.gz" >> ${workingDir}/ICA_AROMA.txt 
    done
done;   

job=${workingDir}/ICA_AROMA.txt

#~~~~~~~~~~~~~~~~~~~~~#
# Run job
#~~~~~~~~~~~~~~~~~~~~~#

    ${dep}/hpcSubmit ${job} 02:00:00 1 8Gb
    echo ""; echo "***"; echo ""; echo "Head submitted commands:"
    head ${job}


## 7) Denoising using ICA

Once ICA AROMA has finished running you will have a directory in the FEAT directory that contains a link to the ICA AROMA report and a text file containing the components which have been labelled as noise. This needs to be edited to reflect what you think should be regressed out. To decide on this you should view each component for each subject and decide whether this is signal or noise (see Griffanti et al., 2017). Keep track of this so we are able to quanity what percentage of components were removed / what our agreement rate with ICA AROMA is etc. Once you have decided which components you wish to remove, edit the test file 'classified_motion_ICs.txt' and save as 'edited_classified_motion_ICs.txt'

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5489418/pdf/main.pdf

`![](/ICA_example.jpg)`


* ICA-MELODIC on unsmoothed data will contain between 70-88% noise components
* Boundaries: noise
* Overlap with WM, CSF: noise
* pathchy numerous small clusters: noise
* sudden changes of oscillation pattern (high freq/very low or pan freq): noise
* Cardiac ~ 1Hz
* Resp ~ 0.3Hz
* Physics artifact (ring): noise
* Dropout regions (orbito-frontal): noise

* Signal (strong peak between 0.01 - 0.1 Hz)


To avoid cleaning real neural singal, approach as innocent untill proven guilty. Spatial maps are the main source of information but use the powerspectrum and timeseries to aid especially when uncertain.   
  
The decision process we suggest to classify a component as signal is:  
    
    1) spatial maps first and foremost need to be plausible, i.e. be locatedin GM, away from the main veins, WM or CSF (with no obviousnoise-related features as described above);

    2) time series should be without sudden, abrupt changes;
    
    3) power spectra should largely show power at low frequency (or at least the low frequency content should be larger than the high frequency one).
    

![title](ICA_example.png)

Once you have decided which components you wish to remove, edit the test file 'classified_motion_ICs.txt' and save as 'edited_classified_motion_ICs.txt. This will become the input for the next step which is to regress out the noise components.

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $project

ICAAROMA=~/c3nl_tools/ICA-AROMA-master/ICA_AROMA.py
aroma="source activate aroma"

#~~~~~~~~~~~~~~~~~~~~~#
# Job
#~~~~~~~~~~~~~~~~~~~~~#

echo -n "" > ${workingDir}/denoise.txt
    
for sub in `ls ${workingDir}/source/`
do
    for ses in `ls ${workingDir}/source/${sub}`
    do 
        echo "${fsl}; fsl_regfilt -i ${workingDir}/source/${sub}/${ses}/func/rest/preICA.feat/filtered_func_data.nii.gz -d ${workingDir}/source/${sub}/${ses}/func/rest/preICA.feat/ICA_AROMA/melodic.ica/melodic_mix -f \`cat ${workingDir}/source/${sub}/${ses}/func/rest/preICA.feat/ICA_AROMA/edited_classified_motion_ICs.txt\` -o ${workingDir}/source/${sub}/${ses}/func/rest/preICA.feat/ICA_AROMA/edited_denoised_data_nonaggr.nii.gz" >> ${workingDir}/denoise.txt  
    done
done;   

job=${workingDir}/denoise.txt

#~~~~~~~~~~~~~~~~~~~~~#
# Run job
#~~~~~~~~~~~~~~~~~~~~~#

    ${dep}/hpcSubmit ${job} 02:00:00 1 8Gb
    echo ""; echo "***"; echo ""; echo "Head submitted commands:"
    head ${job}


## 8) Segment and regress out CSF, white matter and motion parameters


In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $project

ICAAROMA=~/c3nl_tools/ICA-AROMA-master/ICA_AROMA.py
aroma="source activate aroma"

#~~~~~~~~~~~~~~~~~~~~~#
# Job
#~~~~~~~~~~~~~~~~~~~~~#

echo -n "" > ${workingDir}/segmentT1.txt
    
for sub in `ls ${workingDir}/source/`
do
    for ses in `ls ${workingDir}/source/${sub}`
    do 
        inputDir=${workingDir}/source/${sub}/${ses}/anat/T1w
        #Segment T1_brain images into CSF, Grey Matter and White Matter using FSL'S 'FAST'
        echo "fast -o ${inputDir}/${sub}_${ses}_T1w_brain.nii.gz -n 3 -g ${inputDir}/${sub}_${ses}_T1w_brain.nii.gz; fslmaths ${inputDir}/${sub}_${ses}_T1w_brain_pve_0.nii.gz -thr 0.8 ${inputDir}/${sub}_${ses}_T1w_brain_pve_CSF_thrsh.nii.gz; fslmaths ${inputDir}/${sub}_${ses}_T1w_brain_pve_1.nii.gz -thr 0.8 ${inputDir}/${sub}_${ses}_T1w_brain_pve_GM_thrsh.nii.gz; fslmaths ${inputDir}/${sub}_${ses}_T1w_brain_pve_2.nii.gz -thr 0.8 ${inputDir}/${sub}_${ses}_T1w_brain_pve_WM_thrsh.nii.gz " >> ${workingDir}/segmentT1.txt  
    done
done;   

job=${workingDir}/segmentT1.txt

#~~~~~~~~~~~~~~~~~~~~~#
# Run job
#~~~~~~~~~~~~~~~~~~~~~#

    ${dep}/hpcSubmit ${job} 02:00:00 1 8Gb
    echo ""; echo "***"; echo ""; echo "Head submitted commands:"
    head ${job}


#### Extract timeseries for GM, WM, CSF and regress out from functional data

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $project

#~~~~~~~~~~~~~~~~~~~~~#
# Job
#~~~~~~~~~~~~~~~~~~~~~#
    
for sub in `ls ${workingDir}/source/`
do
    for ses in `ls ${workingDir}/source/${sub}`
    do 
    
    echo ${sub}
    #echo "${fsl}; ~/projects/squeezy/scripts/funcRegress -wd ${workingDir} -sub ${sub} -ses ${ses}" >> ${workingDir}/csfRegression.txt

    subDir=${workingDir}/source/${sub}/${ses}
    echo $subDir
    #Transform into functional space and binarise
    echo "Transforming segmentations into functional space "

    flirt -in ${subDir}/anat/T1w/${sub}_${ses}_T1w_brain_pve_CSF_thrsh.nii.gz -applyxfm -init ${subDir}/func/rest/preICA.feat/reg/highres2example_func.mat -out ${subDir}/anat/T1w/${sub}_${ses}_T1_CSF_funcspace.nii.gz -paddingsize 0.0 -interp nearestneighbour -ref ${subDir}/func/rest/preICA.feat/filtered_func_data.nii.gz
    flirt -in ${subDir}/anat/T1w/${sub}_${ses}_T1w_brain_pve_GM_thrsh.nii.gz -applyxfm -init ${subDir}/func/rest/preICA.feat/reg/highres2example_func.mat -out ${subDir}/anat/T1w/${sub}_${ses}_T1_GM_funcspace.nii.gz -paddingsize 0.0 -interp nearestneighbour -ref ${subDir}/func/rest/preICA.feat/filtered_func_data.nii.gz
    flirt -in ${subDir}/anat/T1w/${sub}_${ses}_T1w_brain_pve_WM_thrsh.nii.gz -applyxfm -init ${subDir}/func/rest/preICA.feat/reg/highres2example_func.mat -out ${subDir}/anat/T1w/${sub}_${ses}_T1_WM_funcspace.nii.gz -paddingsize 0.0 -interp nearestneighbour -ref ${subDir}/func/rest/preICA.feat/filtered_func_data.nii.gz

    fslmeants -i ${subDir}/func/rest/preICA.feat/filtered_func_data.nii.gz -o ${subDir}/func/rest/preICA.feat/CSF_timeseries.txt -m ${subDir}/anat/T1w/${sub}_${ses}_T1_CSF_funcspace.nii.gz
    fslmeants -i ${subDir}/func/rest/preICA.feat/filtered_func_data.nii.gz -o ${subDir}/func/rest/preICA.feat/GM_timeseries.txt  -m ${subDir}/anat/T1w/${sub}_${ses}_T1_GM_funcspace.nii.gz
    fslmeants -i ${subDir}/func/rest/preICA.feat/filtered_func_data.nii.gz -o ${subDir}/func/rest/preICA.feat/WM_timeseries.txt  -m ${subDir}/anat/T1w/${sub}_${ses}_T1_WM_funcspace.nii.gz

    #Regress out the CSF and WM timeseries from the cleaned filtered functional data (output from ICA AROMA)
    echo "Regressing out CSF and WM timeseries from ${sub} cleaned data"

    fsl_regfilt -i ${subDir}/func/rest/preICA.feat/ICA_AROMA/edited_denoised_data_nonaggr.nii.gz -d ${subDir}/func/rest/preICA.feat/CSF_timeseries.txt -f "1" -o ${subDir}/func/rest/preICA.feat/edited_denoised_CSFregressed_filtered_func.nii.gz
    fsl_regfilt -i ${subDir}/func/rest/preICA.feat/edited_denoised_CSFregressed_filtered_func.nii.gz -d ${subDir}/func/rest/preICA.feat/WM_timeseries.txt -f "1" -o ${subDir}/func/rest/preICA.feat/edited_denoised_CSF_WM_regressed_filtered_func.nii.gz

    #Put a copy of the cleaned data in the individual analysis directory
    echo "Copying ${sub} cleaned data to individual analysis directory"
    cp ${subDir}/func/rest/preICA.feat/edited_denoised_CSF_WM_regressed_filtered_func.nii.gz ${subDir}/func/rest/${sub}_${ses}_cleaned_filtered_func.nii.gz
    
    done
done


## 9) Lower level FEAT on the cleaned data (postICA.feat)

This time
- input cleaned fMRI data
- do high pass filtering
- no motion correction
- no slice time correction
- registration

In [1]:
%%file /rds/general/user/nbourke/home/templates/postICA_feat.fsf

# FEAT version number
set fmri(version) 6.00

# Are we in MELODIC?
set fmri(inmelodic) 0

# Analysis level
# 1 : First-level analysis
# 2 : Higher-level analysis
set fmri(level) 1

# Which stages to run
# 0 : No first-level analysis (registration and/or group stats only)
# 7 : Full first-level analysis
# 1 : Pre-processing
# 2 : Statistics
set fmri(analysis) 1

# Use relative filenames
set fmri(relative_yn) 0

# Balloon help
set fmri(help_yn) 0

# Run Featwatcher
set fmri(featwatcher_yn) 0

# Cleanup first-level standard-space images
set fmri(sscleanup_yn) 0

# Output directory
set fmri(outputdir) "__MYOUTPUTDIR__"

# TR(s)
set fmri(tr) "__MYTR__"

# Total volumes
set fmri(npts) "__MYNPTS__"

# Delete volumes
set fmri(ndelete) 0

# Perfusion tag/control order
set fmri(tagfirst) 1

# Number of first-level analyses
set fmri(multiple) 1

# Higher-level input type
# 1 : Inputs are lower-level FEAT directories
# 2 : Inputs are cope images from FEAT directories
set fmri(inputtype) 2

# Carry out pre-stats processing?
set fmri(filtering_yn) 1

# Brain/background threshold, %
set fmri(brain_thresh) 10

# Critical z for design efficiency calculation
set fmri(critical_z) 5.3

# Noise level
set fmri(noise) 0.66

# Noise AR(1)
set fmri(noisear) 0.34

# Motion correction
# 0 : None
# 1 : MCFLIRT
set fmri(mc) 0

# Spin-history (currently obsolete)
set fmri(sh_yn) 0

# B0 fieldmap unwarping?
set fmri(regunwarp_yn) 0

# GDC Test
set fmri(gdc) ""

# EPI dwell time (ms)
set fmri(dwell) 0.7

# EPI TE (ms)
set fmri(te) 35

# % Signal loss threshold
set fmri(signallossthresh) 10

# Unwarp direction
set fmri(unwarp_dir) y-

# Slice timing correction
# 0 : None
# 1 : Regular up (0, 1, 2, 3, ...)
# 2 : Regular down
# 3 : Use slice order file
# 4 : Use slice timings file
# 5 : Interleaved (0, 2, 4 ... 1, 3, 5 ... )
set fmri(st) 0

# Slice timings file
set fmri(st_file) ""

# BET brain extraction
set fmri(bet_yn) 0

# Spatial smoothing FWHM (mm)
set fmri(smooth) 0

# Intensity normalization
set fmri(norm_yn) 0

# Perfusion subtraction
set fmri(perfsub_yn) 0

# Highpass temporal filtering
set fmri(temphp_yn) 1

# Lowpass temporal filtering
set fmri(templp_yn) 0

# MELODIC ICA data exploration
set fmri(melodic_yn) 0

# Carry out main stats?
set fmri(stats_yn) 0

# Carry out prewhitening?
set fmri(prewhiten_yn) 1

# Add motion parameters to model
# 0 : No
# 1 : Yes
set fmri(motionevs) 0
set fmri(motionevsbeta) ""
set fmri(scriptevsbeta) ""

# Robust outlier detection in FLAME?
set fmri(robust_yn) 0

# Higher-level modelling
# 3 : Fixed effects
# 0 : Mixed Effects: Simple OLS
# 2 : Mixed Effects: FLAME 1
# 1 : Mixed Effects: FLAME 1+2
set fmri(mixed_yn) 2

# Higher-level permutations
set fmri(randomisePermutations) 5000

# Number of EVs
set fmri(evs_orig) 1
set fmri(evs_real) 2
set fmri(evs_vox) 0

# Number of contrasts
set fmri(ncon_orig) 1
set fmri(ncon_real) 1

# Number of F-tests
set fmri(nftests_orig) 0
set fmri(nftests_real) 0

# Add constant column to design matrix? (obsolete)
set fmri(constcol) 0

# Carry out post-stats steps?
set fmri(poststats_yn) 0

# Pre-threshold masking?
set fmri(threshmask) ""

# Thresholding
# 0 : None
# 1 : Uncorrected
# 2 : Voxel
# 3 : Cluster
set fmri(thresh) 3

# P threshold
set fmri(prob_thresh) 0.05

# Z threshold
set fmri(z_thresh) 3.1

# Z min/max for colour rendering
# 0 : Use actual Z min/max
# 1 : Use preset Z min/max
set fmri(zdisplay) 0

# Z min in colour rendering
set fmri(zmin) 2

# Z max in colour rendering
set fmri(zmax) 8

# Colour rendering type
# 0 : Solid blobs
# 1 : Transparent blobs
set fmri(rendertype) 1

# Background image for higher-level stats overlays
# 1 : Mean highres
# 2 : First highres
# 3 : Mean functional
# 4 : First functional
# 5 : Standard space template
set fmri(bgimage) 1

# Create time series plots
set fmri(tsplot_yn) 1

# Registration to initial structural
set fmri(reginitial_highres_yn) 0

# Search space for registration to initial structural
# 0   : No search
# 90  : Normal search
# 180 : Full search
set fmri(reginitial_highres_search) 90

# Degrees of Freedom for registration to initial structural
set fmri(reginitial_highres_dof) 3

# Registration to main structural
set fmri(reghighres_yn) 1

# Search space for registration to main structural
# 0   : No search
# 90  : Normal search
# 180 : Full search
set fmri(reghighres_search) 90

# Degrees of Freedom for registration to main structural
set fmri(reghighres_dof) BBR

# Registration to standard image?
set fmri(regstandard_yn) 1

# Use alternate reference images?
set fmri(alternateReference_yn) 0

# Standard image
set fmri(regstandard) "/apps/fsl/5.0.10/fsl/data/standard/MNI152_T1_2mm_brain"

# Search space for registration to standard space
# 0   : No search
# 90  : Normal search
# 180 : Full search
set fmri(regstandard_search) 90

# Degrees of Freedom for registration to standard space
set fmri(regstandard_dof) 12

# Do nonlinear registration from structural to standard space?
set fmri(regstandard_nonlinear_yn) 0

# Control nonlinear warp field resolution
set fmri(regstandard_nonlinear_warpres) 10 

# High pass filter cutoff
set fmri(paradigm_hp) 100.0

# Total voxels
set fmri(totalVoxels) 43008000


# Number of lower-level copes feeding into higher-level analysis
set fmri(ncopeinputs) 0

# 4D AVW data or FEAT directory (1)
set feat_files(1) "__MYFMRI__"

# Add confound EVs text file
set fmri(confoundevs) 0

# Subject's structural image for analysis 1
set highres_files(1) "__MYHIGHRES__"

# EV 1 title
set fmri(evtitle1) ""

# Basic waveform shape (EV 1)
# 0 : Square
# 1 : Sinusoid
# 2 : Custom (1 entry per volume)
# 3 : Custom (3 column format)
# 4 : Interaction
# 10 : Empty (all zeros)
set fmri(shape1) 0

# Convolution (EV 1)
# 0 : None
# 1 : Gaussian
# 2 : Gamma
# 3 : Double-Gamma HRF
# 4 : Gamma basis functions
# 5 : Sine basis functions
# 6 : FIR basis functions
set fmri(convolve1) 2

# Convolve phase (EV 1)
set fmri(convolve_phase1) 0

# Apply temporal filtering (EV 1)
set fmri(tempfilt_yn1) 1

# Add temporal derivative (EV 1)
set fmri(deriv_yn1) 1

# Skip (EV 1)
set fmri(skip1) 0

# Off (EV 1)
set fmri(off1) 30

# On (EV 1)
set fmri(on1) 30

# Phase (EV 1)
set fmri(phase1) 0

# Stop (EV 1)
set fmri(stop1) -1

# Gamma sigma (EV 1)
set fmri(gammasigma1) 3

# Gamma delay (EV 1)
set fmri(gammadelay1) 6

# Orthogonalise EV 1 wrt EV 0
set fmri(ortho1.0) 0

# Orthogonalise EV 1 wrt EV 1
set fmri(ortho1.1) 0

# Contrast & F-tests mode
# real : control real EVs
# orig : control original EVs
set fmri(con_mode_old) orig
set fmri(con_mode) orig

# Display images for contrast_real 1
set fmri(conpic_real.1) 1

# Title for contrast_real 1
set fmri(conname_real.1) ""

# Real contrast_real vector 1 element 1
set fmri(con_real1.1) 1

# Real contrast_real vector 1 element 2
set fmri(con_real1.2) 0

# Display images for contrast_orig 1
set fmri(conpic_orig.1) 1

# Title for contrast_orig 1
set fmri(conname_orig.1) ""

# Real contrast_orig vector 1 element 1
set fmri(con_orig1.1) 1

# Contrast masking - use >0 instead of thresholding?
set fmri(conmask_zerothresh_yn) 0

# Do contrast masking at all?
set fmri(conmask1_1) 0

##########################################################
# Now options that don't appear in the GUI

# Alternative (to BETting) mask image
set fmri(alternative_mask) ""

# Initial structural space registration initialisation transform
set fmri(init_initial_highres) ""

# Structural space registration initialisation transform
set fmri(init_highres) ""

# Standard space registration initialisation transform
set fmri(init_standard) ""

# For full FEAT analysis: overwrite existing .feat output dir?
set fmri(overwrite_yn) 0

Writing /rds/general/user/nbourke/home/templates/postICA_feat.fsf


### *Hacked cell, not following the standard data format
Data has been moved at this stage due to the limitation of time in ephemeral and taking a break over christmas

In [8]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $project

${fsl}
#~~~~~~~~~~~~~~~~~~~~~#
# Job
#~~~~~~~~~~~~~~~~~~~~~#

# Path to the template .fsf file (defined above)
template_fsf=${templates}/postICA_feat.fsf;
#template_fsf=/rds/general/user/nbourke/home/templates/postICA_feat.fsf;
#echo $template_fsf
# # Blank commands file for parallelise
echo -n "" > ${workingDir}/clone_postICA_feat_template.txt

conDir=/rds/general/user/nbourke/home/eph/controlTestSet

for sub in `ls ${conDir}`
do        
        # Make a .fsf file which will be individualised to this subject/visit
        fsffile="${conDir}/${sub}/func/rest/${sub}_postICA_feat.fsf"
        
        # Copy the template onto this temporary file
        cp ${template_fsf} ${fsffile} 
    
        fmriInput=${conDir}/${sub}/func/rest/${sub}_cleaned_filtered_func.nii.gz
        t1Input=${conDir}/${sub}/anat/T1w/${sub}_T1w_brain.nii.gz
        outDir=${conDir}/${sub}/func/rest/postICA
        
        
        # This block will replace all the keys in map[key]=value that occur in the .fsf file
        # with their corresponding values 
        declare -A map=()
        map[__MYFMRI__]="${fmriInput}"
        map[__MYHIGHRES__]="${t1Input}"
        map[__MYOUTPUTDIR__]="${outDir}"
        map[__MYTR__]=`fslinfo ${map[__MYFMRI__]} | grep ^pixdim4 | awk '{print $2}'`
        map[__MYNPTS__]=`fslnvols ${map[__MYFMRI__]}`
       
        for j in "${!map[@]}"
        do
            from=$j; to=${map[$j]};
            sed -i "s|$from|$to|g" ${fsffile}
        done

        echo "${fsl}; feat ${fsffile}" >> ${workingDir}/clone_postICA_feat_template.txt
done;

job=${workingDir}/clone_postICA_feat_template.txt 


# pause, check feat files

#~~~~~~~~~~~~~~~~~~~~~#
# Run job
#~~~~~~~~~~~~~~~~~~~~~#

    #${dep}/hpcSubmit ${job} 02:00:00 1 8Gb
    echo ""; echo "***"; echo ""; echo "Head submitted commands:"
    head ${job}


squeezy

***

Head submitted commands:
module load fsl; feat /rds/general/user/nbourke/home/eph/controlTestSet/ses-CIF_CONTROL005/func/rest/ses-CIF_CONTROL005_postICA_feat.fsf
module load fsl; feat /rds/general/user/nbourke/home/eph/controlTestSet/ses-CIF_CONTROL008/func/rest/ses-CIF_CONTROL008_postICA_feat.fsf
module load fsl; feat /rds/general/user/nbourke/home/eph/controlTestSet/ses-CIF_CONTROL009/func/rest/ses-CIF_CONTROL009_postICA_feat.fsf
module load fsl; feat /rds/general/user/nbourke/home/eph/controlTestSet/ses-CIF_CONTROL011/func/rest/ses-CIF_CONTROL011_postICA_feat.fsf
module load fsl; feat /rds/general/user/nbourke/home/eph/controlTestSet/ses-CIF_CONTROL013/func/rest/ses-CIF_CONTROL013_postICA_feat.fsf
module load fsl; feat /rds/general/user/nbourke/home/eph/controlTestSet/ses-CIF_CONTROL014/func/rest/ses-CIF_CONTROL014_postICA_feat.fsf
module load fsl; feat /rds/general/user/nbourke/home/eph/controlTestSet/ses-CIF_CONTROL030/func/rest/ses-CIF_CONTROL030_postICA_feat.fsf
mo

 /home/nbourke/anaconda3/envs/py27/bin /apps/gcc/6.2.0/ /opt/ibutils/bin /rds/general/user/nbourke/home/anaconda3/bin /apps/mrtrix/3.0/bin /apps/ants/2015-02-23/bin/bin /apps/gcc/6.2.0/bin /usr/local/sbin /opt/pbs/bin /usr/lib64/qt-3.3/bin /apps/ants/2015-02-23/bin/ /rds/general/user/nbourke/home/perl5/bin /apps/anaconda3/4.5.12/install


Redundent cell?? (also wrong)
the post ICA direcotry should contain a standard filtered func clean file

In [8]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $project

module load fsl
#~~~~~~~~~~~~~~~~~~~~~#
# Job
#~~~~~~~~~~~~~~~~~~~~~#

conDir=/rds/general/user/nbourke/home/eph/controlTestSet

for sub in `ls ${conDir}`
do     
    dataDir=${conDir}/${sub}/func/rest/postICA.feat
    #mv ${dataDir}/filtered_func_data.nii.gz ${dataDir}/filtered_func_data_clean.nii.gz
    
    #echo $dataDir
    applywarp -r ${dataDir}/reg/standard.nii.gz -i ${dataDir}/filtered_func_data_clean.nii.gz -o ${dataDir}/filtered_func_data_clean_standard.nii.gz --premat=${dataDir}/reg/example_func2standard.mat # -w ${dataDir}/reg/highres2standard_warp.nii.gz   
done;


bash: line 2: $setup: No such file or directory
 /home/nbourke/anaconda3/envs/py27/bin /apps/gcc/6.2.0/ /opt/ibutils/bin /rds/general/user/nbourke/home/anaconda3/bin /apps/mrtrix/3.0/bin /apps/ants/2015-02-23/bin/bin /apps/gcc/6.2.0/bin /usr/local/sbin /opt/pbs/bin /usr/lib64/qt-3.3/bin /apps/ants/2015-02-23/bin/ /rds/general/user/nbourke/home/perl5/bin /apps/anaconda3/4.5.12/install


Thinking about Dual Regression:

- What connectivity measures are we interested in here?

